In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.preprocessing import StandardScaler
import random

In [2]:
datas = '/Users/pkrok/OneDrive/Python/Giannakopoulos/MyData.xlsx'

In [3]:
pd.set_option("display.max_columns", None)
df=pd.read_excel(datas)
df.head(2)
df['logD'] = np.log10(df['diffusivity'])
# df = df.dropna()

# Keep only the columns I need

In [4]:
df2=df[[ 'aperture', 'aperture2', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'ionicRad', 
       'Μ-N_lff', 'Μ-N_kFF', 'ionicRad', 'MetalCharge',
       'apertureAtom_σ', 'apertureAtom_e', 'linker_length1', 'linker_length2',
       'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
       'func1_length', 'func2_length', 'func3_length', 'func1_mass',
       'func2_mass', 'func3_mass', 'func1_charge', 'func2_charge',
       'func3_charge',]]
df2.head()

,aperture,aperture2,size - van der Waals (Å),mass,ascentricF,logD,size - kinetic diameter (Å),ionicRad,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_σ,apertureAtom_e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass2,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,3.92,3.93,2.66,4.002,-0.390,-7.221849,2.60,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,3.92,NaN,2.76,2.010,-0.217,-7.301030,2.89,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
2,3.92,NaN,2.94,31.999,0.022,-8.000000,3.46,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
3,3.92,3.97,3.24,44.010,0.225,-8.397940,3.30,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
4,3.92,NaN,3.13,28.000,0.037,-8.301030,3.64,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001


In [5]:
df2=df2.rename(columns={'size - van der Waals (Å)':'diameter', 'size - kinetic diameter (Å)':'kdiameter', 'apertureAtom_e':'e' })

In [6]:
df2 = df2.dropna()
df2=df2.reset_index(drop=True)
df2.head(2)

,aperture,aperture2,diameter,mass,ascentricF,logD,kdiameter,ionicRad,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_σ,e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass2,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,3.92,3.93,2.66,4.002,-0.390,-7.221849,2.6,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,3.92,3.97,3.24,44.010,0.225,-8.397940,3.3,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001


# Create my dataset x and y

In [7]:
x = np.asanyarray(df2[['aperture', 'apertureAtom_σ', 'diameter', 'kdiameter', 'ascentricF', 'ionicRad','mass', 'linker_length1', 'linker_length2',
       'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
       'func1_length', 'func2_length', 'func3_length', 'func1_mass',
       'func2_mass', 'func3_mass','ionicRad','e','Μ-N_lff', 'Μ-N_kFF', 'ionicRad', 'MetalCharge','func1_charge', 'func2_charge',
       'func3_charge' ]])
y = np.asanyarray(df2[['logD']])

# Scale the Data

In [8]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

# Create GridSeachCV for NN regressor

In [ ]:
# parameters= {'hidden_layer_sizes': [n for n in range(10,600)],
#      'solver': ['lbfgs', 'sgd', 'adam'],
#      'max_iter': [n for n in range(100,8000)]}

parameters= {'hidden_layer_sizes': [n for n in range(100,300, 100)],
     'solver': ['adam','lbfgs', 'sgd' ],
     'max_iter': [n for n in range(5000, 6000, 1000)]}

NNregr = MLPRegressor(random_state=1)

In [ ]:
nn_cv = GridSearchCV(NNregr, parameters, cv=10)
nn_cv.fit(x, y.ravel())

In [ ]:
print("tuned hpyerparameters :(best parameters) ", nn_cv.best_params_)
print("accuracy :",nn_cv.best_score_)

# GridSeachCV with tuples

In [9]:
import itertools

bigLayer = [n for n in range(10,61,10)]
middleLayers = [n for n in range(10,51,10)]
smallLayer = [n for n in range(10,41,10)]
                             
myProduct = list(itertools.product(bigLayer, middleLayers, middleLayers, middleLayers, middleLayers, smallLayer))
# myProduct

In [10]:
parameters= {'hidden_layer_sizes': tuple(myProduct),
     'solver': ['adam','lbfgs', 'sgd' ],
     'max_iter': [n for n in range(3000, 7000, 1000)]}

NNregr = MLPRegressor(random_state=1)

In [11]:
nn_cv = GridSearchCV(NNregr, parameters)
nn_cv.fit(x, y.ravel())

C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

KeyboardInterrupt: 

In [ ]:
print("tuned hpyerparameters :(best parameters) ", nn_cv.best_params_)
print("accuracy :",nn_cv.best_score_)